In [1]:
import os
import re
import codecs
import json
from datetime import datetime

In [6]:
def rename_data_file(**kwargs):
    """
    Funcao que recebe o nome do arquivo original e o renomeia.
    
    Args:
        data_file(str): Nome do arquivo original.
            Ex.: '20110101_AC.txt'
        file_ext(str): Tipo de extensao a ser adicionado ao nome do arquivo.
            Ex.: '.csv'
    
    Returns:
        string: Nome do arquivo modificado.
            Ex.: '20110101_CNPJ.csv'
    """
    # Expressao Regular para:
    # Identificar a data de referencia no nome do arquivo
    date_reference_re = re.compile(r'^[0-9]+')
    # Identificar o assunto no nome do arquivo
    file_subject_re = re.compile(r'_([\w]+)\.')
    
    # Identifica a data de referencia do arquivo
    date_reference = re.search(date_reference_re, kwargs['data_file']).group()
    # Identifica o assunto do arquivo
    subject = re.search(file_subject_re, kwargs['data_file']).group(1)
    
    # Define o novo nome do arquivo
    renamed_file = date_reference + '_' + subject + kwargs['file_ext']
    
    return renamed_file

In [7]:
def json_to_csv(**kwargs):
    """
    Le o arquivo JSON de entrada e converte para um arquivo de saida CSV.
    
    Args:
        in_data_path(str): Caminho completo do arquivo de entrada.
        out_data_dir(str): Caminho da pasta para armazenamento do arquivo de saida.
        data_file(str): Nome do arquivo de entrada.
        delimiter(str): Delimitador de campos utilizado no arquivo de saida
        file_ext(str): Extensao do arquivo de saida
        encoding(str): Codificacao do arquivo de saida
    """
    
    # Nome do arquivo de saida
    kwargs['out_data_file'] = rename_data_file(**kwargs)
    # Caminho completo do arquivo de saida
    kwargs['out_data_path'] = os.path.join(kwargs['out_data_dir'], kwargs['out_data_file'])
    
    # Log: Mensagem de inicio da conversao
    print(str(datetime.now()) + ': Conversao do arquivo ' + kwargs['data_file'] + ' iniciada.')
    
    # Le o arquivo de entrada
    with codecs.open(kwargs['in_data_path'], 'r', encoding = kwargs['encoding']) as reader:
        
        # Prepara o arquivo json
        data = json.load(reader)
        
        # Define o header para o arquivo de saida separado por tabulacao
        kwargs['header'] = 'id_municipio\tnm_municipio\tid_microrregiao\tnm_microrregiao\tid_mesorregiao\tnm_mesorregiao\tid_uf\tsg_uf\tnm_uf\tid_regiao\tsg_regiao\tnm_regiao\n'
        
        # Prepara o arquivo de saida
        with codecs.open(kwargs['out_data_path'], 'w', encoding = kwargs['encoding']) as writer:
            
            # Escreve o cabecalho no arquivo de saida
            writer.write(kwargs['header'])
            
            # Para cada objeto no arquivo de entrada
            for d in data:
            
                # Escreve o objeto no formato tabular no arquivo de saida
                    writer.write(
                                                str(d['id'])                                                 # id_municipio
                        + kwargs['delimiter'] + d['nome']                                                    # nm_municipio
                        + kwargs['delimiter'] + str(d['microrregiao']['id'])                                 # id_microrregiao
                        + kwargs['delimiter'] + d['microrregiao']['nome']                                    # nm_microrregiao
                        + kwargs['delimiter'] + str(d['microrregiao']['mesorregiao']['id'])                  # id_mesorregiao
                        + kwargs['delimiter'] + d['microrregiao']['mesorregiao']['nome']                     # nm_mesorregiao
                        + kwargs['delimiter'] + str(d['microrregiao']['mesorregiao']['UF']['id'])            # id_uf
                        + kwargs['delimiter'] + d['microrregiao']['mesorregiao']['UF']['sigla']              # sg_uf
                        + kwargs['delimiter'] + d['microrregiao']['mesorregiao']['UF']['nome']               # nm_uf
                        + kwargs['delimiter'] + str(d['microrregiao']['mesorregiao']['UF']['regiao']['id'])  # id_regiao
                        + kwargs['delimiter'] + d['microrregiao']['mesorregiao']['UF']['regiao']['sigla']    # sg_regiao
                        + kwargs['delimiter'] + d['microrregiao']['mesorregiao']['UF']['regiao']['nome']     # nm_regiao
                        + '\r\n'
                    )
                    
    # Log: Resultado da verificacao dos registros
    print(str(datetime.now()) + ': Conversao do arquivo JSON para CSV finalizada.')

In [8]:
def organize_files():
    """
    Para cada arquivo de origem, e verificado se a quantidade de colunas de cada registro corresponde
    com a quantidade de colunas do cabecalho do arquivo.        
    """
    
    # Prepara o dicionario de variaveis (kwargs = keyworded arguments)
    kwargs = {}
    
    # Diretorio de armazenamento dos arquivos originais
    kwargs['in_data_dir'] = '..\\data\\02-cleaned'
    
    # Diretorio de armazenamento dos arquivos tratados
    kwargs['out_data_dir'] = '..\\data\\03-organized'
    
    # Lista dos arquivos originais 
    kwargs['data_files'] = os.listdir(kwargs['in_data_dir'])
    
    # Parametros de organizacao dos arquivos delimitados
    kwargs['delimiter'] = '\t'
    kwargs['file_ext'] = '.csv'
    kwargs['encoding'] = 'utf-8'
    
    # Log: Mensagem de inicio do processo 
    print(str(datetime.now()) + ': Organizacao dos arquivos iniciada.')
    
    # Para cada arquivo na lista de arquivos originais
    for file in kwargs['data_files']:
        
        # Define o caminho completo de acesso e escrita dos arquivos
        kwargs['data_file'] = file
        kwargs['in_data_path'] = os.path.join(kwargs['in_data_dir'], kwargs['data_file'])
        kwargs['out_data_path'] = os.path.join(kwargs['out_data_dir'], kwargs['data_file'])
        
        # Corrige registros com menos colunas que o esperado
        json_to_csv(**kwargs)
      
    # Log: Mensagem de finalizacao do processo de limpeza
    print(str(datetime.now()) + ': Organizacao dos arquivos finalizada.')

In [9]:
def main():
    """
    Organiza os dados de Localidades - Municipio
    Origem: IBGE
    """
    
    ### Organiza os arquivos
    organize_files()

if __name__ == '__main__':
    main()

2019-08-23 10:18:05.928256: Organizacao dos arquivos iniciada.
2019-08-23 10:18:05.928256: Conversao do arquivo 20190823_municipios.json iniciada.
2019-08-23 10:18:06.053858: Conversao do arquivo JSON para CSV finalizada.
2019-08-23 10:18:06.069781: Organizacao dos arquivos finalizada.
